In [8]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import pyodbc as odbc
from k_means_constrained import KMeansConstrained
import math

# Import Data From SQL
#### Property Data: ModelTestBed..msrportfolio
#### Tech Data: IRSPublish..tv_technicianAddress

In [9]:
Driver_name = 'SQL Server Native Client 11.0'
Server_Name = 'DFILSQL02.insightlabs.amherst.com'
Database_Name = 'IRSPublish'


cnxn = odbc.connect("Driver={SQL Server Native Client 11.0};"
                    "Server=DFILSQL02.insightlabs.amherst.com;"
                    "Trusted_Connection=yes;")

In [10]:
query_lat_long_data = '''
SELECT [PropertyID] as propertyid
      ,[BranchName] as branchname
      ,[State] as state
      ,[CensusTractGeoId]
      ,[latitude] as property_lat
      ,[longitude] as property_long
      ,[SUBDIVISION_NAME] as subdivision_name
  FROM [ModelTestBed].[dbo].[MSRportfolio]
  where [Still Under Mgmt]='Yes'
  and BranchName like '%st louis%'
'''

raw_prop_data = pd.io.sql.read_sql(query_lat_long_data,cnxn)

lat_long_df = raw_prop_data.copy()
# lat_long_df = lat_long_df[lat_long_df['state'] == 'MO']
lat_long_df

C:\Users\cpaige\AppData\Local\Temp\ipykernel_3620\2299689251.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  raw_prop_data = pd.io.sql.read_sql(query_lat_long_data,cnxn)


DatabaseError: Execution failed on sql '
SELECT [PropertyID] as propertyid
      ,[BranchName] as branchname
      ,[State] as state
      ,[CensusTractGeoId]
      ,[latitude] as property_lat
      ,[longitude] as property_long
      ,[SUBDIVISION_NAME] as subdivision_name
  FROM [ModelTestBed].[dbo].[MSRportfolio]
  where [Still Under Mgmt]='Yes'
  and BranchName like '%st louis%'
': ('42S22', "[42S22] [Microsoft][SQL Server Native Client 11.0][SQL Server]Invalid column name 'SUBDIVISION_NAME'. (207) (SQLExecDirectW)")

In [ ]:
lat_long_df.drop(lat_long_df.loc[lat_long_df['property_long'].isnull()].index, inplace=True)
lat_long_df.drop(lat_long_df.loc[lat_long_df['property_lat'].isnull()].index, inplace=True)

In [11]:
query_techData = """
select distinct ta.branchname, ta.[Primary Home Address - State] as state  ,ta.Worker as tech_name, ta.[Email - Primary Work] as tech_email ,ta.TechZipCode as tech_zip, ZipShp.LAT as tech_lat ,ZipShp.lon as tech_long
from IRSPublish..tv_technicianAddress ta
join thirdpartydata..Shp_ZipCode_dt ZipShp
on ta.TechZipCode=ZipShp.ZipCode
join modeltestbed..msrportfolio msr
on msr.branchname like ta.Branchname+'%'
and [Still Under Mgmt]='Yes'
join thirdpartydata..Shp_Blocks_dt blkShp
on blkShp.fips+blkShp.censusTract+blkshp.BlockGroup=msr.census_block
where ta.branchName = 'St Louis'

"""

raw_tech_data = pd.io.sql.read_sql(query_techData,cnxn)

tech_df = raw_tech_data.copy()
# tech_df = tech_df[tech_df['state'] == 'Missouri']
tech_df

C:\Users\cpaige\AppData\Local\Temp\ipykernel_3620\1025026994.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  raw_tech_data = pd.io.sql.read_sql(query_techData,cnxn)


DatabaseError: Execution failed on sql '
select distinct ta.branchname, ta.[Primary Home Address - State] as state  ,ta.Worker as tech_name, ta.[Email - Primary Work] as tech_email ,ta.TechZipCode as tech_zip, ZipShp.LAT as tech_lat ,ZipShp.lon as tech_long
from IRSPublish..tv_technicianAddress ta
join thirdpartydata..Shp_ZipCode_dt ZipShp
on ta.TechZipCode=ZipShp.ZipCode
join modeltestbed..msrportfolio msr
on msr.branchname like ta.Branchname+'%'
and [Still Under Mgmt]='Yes'
join thirdpartydata..Shp_Blocks_dt blkShp
on blkShp.fips+blkShp.censusTract+blkshp.BlockGroup=msr.census_block
where ta.branchName = 'St Louis'

': ('42S22', "[42S22] [Microsoft][SQL Server Native Client 11.0][SQL Server]Invalid column name 'census_block'. (207) (SQLExecDirectW)")

# K-Means Clustering Assignments

### mini-pod: n_clusters = # of techs
### big-pod: n_clusters = 2

In [ ]:
miniPod_kmeans = KMeansConstrained(n_clusters=len(tech_df), size_min=round(len(lat_long_df) / len(tech_df)) - 3)

lat_long_df['miniPod_id'] = miniPod_kmeans.fit_predict(lat_long_df[['property_lat', 'property_long']])

lat_long_df['miniPod_id'] = lat_long_df['miniPod_id'] + 1

In [ ]:
miniPod_centers = miniPod_kmeans.cluster_centers_

In [ ]:
miniPod_labels = miniPod_kmeans.predict(lat_long_df[['property_lat', 'property_long']])
miniPod_labels = miniPod_labels + 1

In [ ]:
lat_long_df['miniPod_id'].value_counts().sort_index(ascending=True)

In [ ]:
lat_long_df.plot.scatter(x='property_lat', y='property_long', c=miniPod_labels, s=50, cmap= 'seismic')
plt.scatter(miniPod_centers[:, 0], miniPod_centers[:, 1], c='black', s=75, alpha=1)

In [ ]:
n_clus = math.ceil(len(lat_long_df)/1500)

In [ ]:
bigPod_kmeans = KMeansConstrained(n_clusters=n_clus, size_min=round(len(lat_long_df) / n_clus) - 3)

lat_long_df['bigPod_id'] = bigPod_kmeans.fit_predict(lat_long_df[['property_lat', 'property_long']])

lat_long_df['bigPod_id'] = lat_long_df['bigPod_id'] + 1

In [ ]:
bigPod_centers = bigPod_kmeans.cluster_centers_

In [ ]:
bigPod_labels = bigPod_kmeans.predict(lat_long_df[['property_lat', 'property_long']])
bigPod_labels = bigPod_labels + 1

In [ ]:
lat_long_df['bigPod_id'].value_counts().sort_index(ascending=True)

In [ ]:
lat_long_df.plot.scatter(x='property_lat', y='property_long', c=bigPod_labels, s=50, cmap= 'seismic')
plt.scatter(bigPod_centers[:, 0], bigPod_centers[:, 1], c='black', s=75, alpha=1)

In [ ]:
collected_df = lat_long_df.copy()

In [ ]:
def censusTractSplit(collected_df):

    collected_df['fips'] = [x[0:5] for x in collected_df['CensusTractGeoId'].astype(str)]

    df_assign = collected_df.groupby(['fips','subdivision_name'])['miniPod_id'].nunique().to_frame().reset_index()

    df_assign = df_assign[df_assign['miniPod_id']>1]
    df_assign = df_assign.rename(columns={'miniPod_id':'nMiniPods'})

    N_prop = collected_df.groupby(['fips','subdivision_name','miniPod_id'])['propertyid'].count().to_frame().reset_index()
    N_prop = N_prop.rename(columns = {'propertyid':'nProp'})

    to_choose = N_prop.merge(df_assign, left_on=['fips','subdivision_name'], right_on= ['fips','subdivision_name'])

    to_choose['miniPod_rank'] =  to_choose.groupby(['fips','subdivision_name'])['nProp'].rank(method = 'first',ascending=False)

    to_choose = to_choose[to_choose['miniPod_rank']==1][['fips','subdivision_name','miniPod_id']]

    #now we just merge back
    assigned = collected_df.merge(to_choose,how='left', left_on=['fips','subdivision_name'], right_on=['fips','subdivision_name'])
    assigned = assigned.rename(columns={'miniPod_id_x':'prevMiniPodID','miniPod_id_y':'miniPod_id'})

    new_assigned_df = assigned[assigned['miniPod_id'].notnull()].drop('prevMiniPodID', axis=1)

    lst = list(new_assigned_df['propertyid'])

    collected_df.drop(collected_df.loc[collected_df['propertyid'].isin(lst)].index, inplace=True)

    collected_df = pd.concat([collected_df, new_assigned_df])

    collected_df = collected_df.drop('fips', axis=1)

    collected_df['propertyid'] = collected_df['propertyid'].astype(int)
    collected_df['miniPod_id'] = collected_df['miniPod_id'].astype(int)
    collected_df['bigPod_id'] = collected_df['bigPod_id'].astype(int)


    return collected_df


In [ ]:
collected_df = censusTractSplit(collected_df)

In [ ]:
def same_bigPod(df):

    df_assign = df.groupby(['miniPod_id'])['bigPod_id'].nunique().to_frame().reset_index().rename(columns={'bigPod_id': 'nBigPods'})

    df_assign = df_assign[df_assign['nBigPods']>1]

    n_prop = df.groupby(['miniPod_id', 'bigPod_id'])['propertyid'].count().to_frame().reset_index().rename(columns = {'propertyid':'nProp'})

    to_choose = n_prop.merge(df_assign, left_on='miniPod_id', right_on='miniPod_id')

    to_choose['miniPod_rank'] = to_choose.groupby('miniPod_id')['nProp'].rank(method='first', ascending=False)

    to_choose = to_choose[to_choose['miniPod_rank'] ==1][['miniPod_id', 'bigPod_id']]

    assigned = df.merge(to_choose, how='left', left_on='miniPod_id', right_on='miniPod_id')
    assigned = assigned.rename(columns={'bigPod_id_x':'prevBigPodID','bigPod_id_y':'bigPod_id'})

    new_assigned = assigned[assigned['bigPod_id'].notnull()].drop('prevBigPodID', axis=1)

    lst = list(new_assigned['propertyid'])

    df.drop(df.loc[df['propertyid'].isin(lst)].index, inplace=True)

    new_df = new_assigned[['propertyid', 'branchname', 'CensusTractGeoId', 'property_lat', 'property_long', 'subdivision_name', 'miniPod_id', 'bigPod_id']]

    final_df = pd.concat([df, new_df])

    return final_df

In [ ]:
collected_df = same_bigPod(collected_df)

In [ ]:
def get_centers():
    centers_df = pd.DataFrame(miniPod_centers)
    centers_df.rename(columns={0:'miniPod_lat', 1:'miniPod_long'}, inplace=True)

    labels_df = pd.DataFrame(set(miniPod_labels))
    labels_df = labels_df.rename(columns={0:'clusterLabel'})

    centers_df['miniPod_id'] = labels_df['clusterLabel']

    centers_df['miniPod_id'] = centers_df['miniPod_id']

    return centers_df

In [ ]:
collected_df = collected_df.merge(get_centers(), how='left', on='miniPod_id')

In [ ]:
def get_bigPod_center():



    pod_centers = pd.DataFrame(bigPod_centers)
    pod_centers['bigPod_id'] = np.unique(bigPod_labels)
    pod_centers.rename(columns={0:'bigPod_lat', 1:'bigPod_long'}, inplace=True)

    return pod_centers

In [ ]:
collected_df = collected_df.merge(get_bigPod_center(), how='left', on='bigPod_id')

In [ ]:
import openrouteservice as ors

client = ors.Client(base_url='http://localhost:8080/ors')

def miniPodcenter2prop(row):
    index, value = row
    try:
        coords = ((value["miniPod_long"],value["miniPod_lat"]),(value['property_long'],value['property_lat']))
        routes = client.directions(coords, optimize_waypoints=True ,radiuses=50000)
        result = {
            'propertyid': value['propertyid'],
            'branchname': value['branchname'],
            'CensusTractGeoId' : value['CensusTractGeoId'],
            'subdivision_name': value['subdivision_name'],
            'property_lat': value['property_lat'],
            'property_long': value['property_long'],
            'miniPod_id': value['miniPod_id'],
            'bigPod_id': value['bigPod_id'],
            'miniPod_lat':value['miniPod_lat'],
            'miniPod_long': value['miniPod_long'],
            'bigPod_lat':value['bigPod_lat'],
            'bigPod_long': value['bigPod_long'],
            'miniPodcenter2prop_driving_distance(meters)': routes['routes'][0]['summary']['distance'],
            'miniPodcenter2prop_driving_time(secs)': routes['routes'][0]['summary']['duration']
        }
    except:
        result = {'invalid_row': index}
    return result


from multiprocessing.pool import ThreadPool

pool = ThreadPool(14)
results = pool.map(miniPodcenter2prop, collected_df.iterrows())
pool.close()

In [ ]:
collected_df = pd.DataFrame([x_ for x_ in results if not x_.get('invalid_row')])

In [ ]:
def bigPododcenter2prop(row):
    index, value = row
    try:
        coords = ((value["bigPod_long"],value["bigPod_lat"]),(value['property_long'],value['property_lat']))
        routes = client.directions(coords, optimize_waypoints=True ,radiuses=50000)
        result = {
            'propertyid': value['propertyid'],
            'branchname': value['branchname'],
            'CensusTractGeoId' : value['CensusTractGeoId'],
            'subdivision_name': value['subdivision_name'],
            'property_lat': value['property_lat'],
            'property_long': value['property_long'],
            'miniPod_id': value['miniPod_id'],
            'bigPod_id': value['bigPod_id'],
            'miniPod_lat':value['miniPod_lat'],
            'miniPod_long': value['miniPod_long'],
            'bigPod_lat': value['bigPod_lat'],
            'bigPod_long': value['bigPod_long'],
            'miniPodcenter2prop_driving_distance(meters)': value['miniPodcenter2prop_driving_distance(meters)'],
            'miniPodcenter2prop_driving_time(secs)': value['miniPodcenter2prop_driving_time(secs)'],
            'bigPodcenter2prop_driving_distance(meters)': routes['routes'][0]['summary']['distance'],
            'bigPodcenter2prop_driving_time(secs)': routes['routes'][0]['summary']['duration']
        }
    except:
        result = {'invalid_row': index}
    return result


from multiprocessing.pool import ThreadPool

pool = ThreadPool(14)
results = pool.map(bigPododcenter2prop, collected_df.iterrows())
pool.close()

In [ ]:
collected_df = pd.DataFrame([x_ for x_ in results if not x_.get('invalid_row')])

# Tech Assignment and Optimization

In [ ]:
tech2center = tech_df.merge(get_centers(), how='cross')

In [ ]:
def tech2center_(row):
    index, value = row
    try:
        coords = ((value["tech_long"],value["tech_lat"]),(value['miniPod_long'],value['miniPod_lat']))
        routes = client.directions(coords, optimize_waypoints=True ,radiuses=15000)
        result = {
            'tech_zip': value['tech_zip'],
            'tech_name': value['tech_name'],
            'tech_email': value['tech_email'],
            'tech_lat': value['tech_lat'],
            'tech_long': value['tech_long'],
            'miniPod_lat': value['miniPod_lat'],
            'miniPod_long': value['miniPod_long'],
            'miniPod_id':value['miniPod_id'],
            'driving_distance': routes['routes'][0]['summary']['distance'],
            'driving_time': routes['routes'][0]['summary']['duration']
        }
    except:
        result = {'invalid_row': index}
    return result


from multiprocessing.pool import ThreadPool

pool = ThreadPool(14)
results = pool.map(tech2center_, tech2center.iterrows())
pool.close()

In [ ]:
tech2center_df = pd.DataFrame([x_ for x_ in results if not x_.get('invalid_row')])

In [ ]:
tech2center_final = tech2center_df.groupby(['tech_name', 'miniPod_id'])['driving_time'].sum().to_frame().reset_index().sort_values(['tech_name','miniPod_id'])

In [ ]:
tech_assigned = tech2center_final.copy()

In [ ]:
tech2center_final = tech2center_final.set_index(['miniPod_id','tech_name'],drop = True).unstack('miniPod_id')

In [ ]:
tech2center_final.columns = tech2center_final.columns.droplevel(0)

In [ ]:
from ortools.graph.python import linear_sum_assignment
import numpy as np

drivingTime_array = tech2center_final.to_numpy()

In [ ]:
# Let's transform this into 3 parallel vectors (start_nodes, end_nodes, arc_costs)
end_nodes_unraveled, start_nodes_unraveled = np.meshgrid(np.arange(drivingTime_array.shape[1]),np.arange(drivingTime_array.shape[0]))
start_nodes = start_nodes_unraveled.ravel()
end_nodes = end_nodes_unraveled.ravel()
arc_costs = drivingTime_array.ravel()

In [ ]:
assignment = linear_sum_assignment.SimpleLinearSumAssignment()

In [ ]:
assignment.add_arcs_with_cost(start_nodes, end_nodes, arc_costs)

In [ ]:
status = assignment.solve()

In [ ]:
if status == assignment.OPTIMAL:
    print(f'Total driving time for all Techs: {assignment.optimal_cost()}', 'mins\n')
    for i in range(0, assignment.num_nodes()):
        print(f'Technician {i} assigned to mini pod: {assignment.right_mate(i)}' +
              f'  (Driving Time(mins) = {assignment.assignment_cost(i)})')
elif status == assignment.INFEASIBLE:
    print('No assignment is possible.')
elif status == assignment.POSSIBLE_OVERFLOW:
    print(
        'Some input costs are too large and may cause an integer overflow.')

In [ ]:
optimizedTime_lst = []
cluster_lst = []
for i in range(0, assignment.num_nodes()):
    optimizedTime_lst.append(assignment.assignment_cost(i))
    cluster_lst.append(assignment.right_mate(i))

In [ ]:
tech_assigned = tech_assigned.merge(tech2center_df[['tech_name','tech_lat', 'tech_long', 'miniPod_lat', 'miniPod_long']])

In [ ]:
lst = []
lst2 = []
df = pd.DataFrame()
for i in tech_df['tech_name']:
    lat = tech_df.loc[tech_df['tech_name'] == i, 'tech_lat'].iloc[0]
    long = tech_df.loc[tech_df['tech_name'] == i, 'tech_long'].iloc[0]
    lst.append(lat)
    lst2.append(long)


df['tech_lat'] = lst
df['tech_long'] = lst2

In [ ]:
optimizedTime_techAssignment = pd.DataFrame({
    'tech_name': list(tech_assigned['tech_name'].unique()),
    'tech_lat': list(df['tech_lat']),
    'tech_long': list(df['tech_long']),
    'miniPod_lat': list(tech_assigned['miniPod_lat'].unique()),
    'miniPod_long': list(tech_assigned['miniPod_long'].unique()),
    'miniPod_id': cluster_lst,
    'optimizedDrivingTimes(mins)': optimizedTime_lst,
})

optimizedTime_techAssignment['miniPod_id'] = optimizedTime_techAssignment['miniPod_id'] + 1

In [ ]:
tech_assignment = optimizedTime_techAssignment.copy()

In [ ]:
tech_assigned = tech_assignment.merge(tech_df[['tech_name', 'tech_zip', 'tech_email']], how='left', on='tech_name')

In [ ]:
collected_df = collected_df.merge(tech_assigned[['miniPod_id','tech_name','tech_email', 'tech_zip', 'tech_lat', 'tech_long']], how='left', on='miniPod_id')

In [ ]:
def tech2prop(row):
    index, value = row
    try:
        coords = ((value["tech_long"],value["tech_lat"]),(value['property_long'],value['property_lat']))
        routes = client.directions(coords, optimize_waypoints=True ,radiuses=50000)
        result = {
            'propertyid': value['propertyid'],
            'branchname': value['branchname'],
            'subdivision_name': value['subdivision_name'],
            'property_lat': value['property_lat'],
            'property_long': value['property_long'],
            'miniPod_id': value['miniPod_id'],
            'bigPod_id': value['bigPod_id'],
            'miniPod_lat':value['miniPod_lat'],
            'miniPod_long': value['miniPod_long'],
            'bigPod_lat': value['bigPod_lat'],
            'bigPod_long': value['bigPod_long'],
            'miniPodcenter2prop_driving_distance(meters)': value['miniPodcenter2prop_driving_distance(meters)'],
            'miniPodcenter2prop_driving_time(secs)': value['miniPodcenter2prop_driving_time(secs)'],
            'bigPodcenter2prop_driving_distance(meters)': value['bigPodcenter2prop_driving_distance(meters)'],
            'bigPodcenter2prop_driving_time(secs)': value['bigPodcenter2prop_driving_time(secs)'],
            'tech_name': value['tech_name'],
            'tech_zip': value['tech_zip'],
            'tech_email': value['tech_email'],
            'tech_lat': value['tech_lat'],
            'tech_long': value['tech_long'],
            'techZip2prop_driving_distance(meters)':routes['routes'][0]['summary']['distance'],
            'techZip2prop_driving_time(secs)': routes['routes'][0]['summary']['duration']
        }
    except:
        result = {'invalid_row': index}
    return result


from multiprocessing.pool import ThreadPool

pool = ThreadPool(14)
results = pool.map(tech2prop, collected_df.iterrows())
pool.close()

In [ ]:
collected_df = pd.DataFrame([x_ for x_ in results if not x_.get('invalid_row')])

In [ ]:
def tech2minipod(row):
    index, value = row
    try:
        coords = ((value["tech_long"],value["tech_lat"]),(value['miniPod_long'],value['miniPod_lat']))
        routes = client.directions(coords, optimize_waypoints=True ,radiuses=50000)
        result = {
            'propertyid': value['propertyid'],
            'branchname': value['branchname'],
            'subdivision_name': value['subdivision_name'],
            'property_latitude': value['property_lat'],
            'property_longitude': value['property_long'],
            'miniPod_id': value['miniPod_id'],
            'bigPod_id': value['bigPod_id'],
            'miniPod_lat':value['miniPod_lat'],
            'miniPod_long': value['miniPod_long'],
            'bigPod_lat': value['bigPod_lat'],
            'bigPod_long': value['bigPod_long'],
            'miniPod2prop_driving_distance(meters)': value['miniPodcenter2prop_driving_distance(meters)'],
            'miniPod2prop_driving_time(secs)': value['miniPodcenter2prop_driving_time(secs)'],
            'bigPod2prop_driving_distance(meters)': value['bigPodcenter2prop_driving_distance(meters)'],
            'bigPod2prop_driving_time(secs)': value['bigPodcenter2prop_driving_time(secs)'],
            'tech_name': value['tech_name'],
            'tech_zip': value['tech_zip'],
            'tech_email': value['tech_email'],
            'tech_lat': value['tech_lat'],
            'tech_long': value['tech_long'],
            'techZip2prop_driving_distance(meters)':value['techZip2prop_driving_distance(meters)'],
            'techZip2prop_driving_time(secs)': value['techZip2prop_driving_time(secs)'],
            'techZip2miniPod_driving_distance(meters)':routes['routes'][0]['summary']['distance'],
            'techZip2miniPod_driving_time(secs)': routes['routes'][0]['summary']['duration']
        }
    except:
        result = {'invalid_row': index}
    return result


from multiprocessing.pool import ThreadPool

pool = ThreadPool(14)
results = pool.map(tech2minipod, collected_df.iterrows())
pool.close()

In [ ]:
collected_final = pd.DataFrame([x_ for x_ in results if not x_.get('invalid_row')])

In [ ]:
collected_final = collected_final[['propertyid',
                                    'branchname',
                                    'subdivision_name',
                                    'property_latitude',
                                    'property_longitude',
                                    'tech_lat',
                                    'tech_long',
                                    'bigPod_lat',
                                    'bigPod_long',
                                    'miniPod_lat',
                                    'miniPod_long',
                                    'miniPod_id',
                                    'bigPod_id',
                                    'bigPod2prop_driving_distance(meters)',
                                    'bigPod2prop_driving_time(secs)',
                                    'miniPod2prop_driving_distance(meters)',
                                    'miniPod2prop_driving_time(secs)',
                                    'techZip2prop_driving_distance(meters)',
                                    'techZip2prop_driving_time(secs)',
                                    'techZip2miniPod_driving_distance(meters)',
                                    'techZip2miniPod_driving_time(secs)',
                                    'tech_zip',
                                    'tech_name',
                                    'tech_email'
                                   ]]

collected_final['bigPod_id'] = collected_final['bigPod_id'].astype(int)

In [ ]:
collected_final.groupby(['tech_name', 'miniPod_id']).agg({'techZip2miniPod_driving_time(secs)':'mean'})

In [ ]:
collected_final

In [ ]:
# collected_final.to_csv('stLouis_Assigned.csv', index=False)